## Orchestration delays investigation
This notebook makes various kusto queries and produces a report.


In [ ]:
# These are just defaults will be overwritten if you use https://github.com/yaananth/nimport
su="tfs-cus-1"
start="2019-07-20T16:00:00.0000000Z"
end="2019-07-20T16:33:36.0000000Z"
url="https://notebooksv2.azure.com/yaananth/projects/06OasuNRs6rK/delays.ipynb"
baseUrl="https://notebooksv2.azure.com/yaananth/projects/06OasuNRs6rK"
service="tfs"
hub="Build"

In [ ]:
# Import the things we use

# Note you can also use kql https://docs.microsoft.com/en-us/azure/data-explorer/kqlmagic
# %kql is single line magic
# %%kql is cell magic

# https://nbviewer.jupyter.org/github/ipython/ipython/blob/4.0.x/examples/IPython%20Kernel/Rich%20Output.ipynb#HTML
# https://ipython.readthedocs.io/en/stable/inte/magics.html
from IPython.display import display, HTML, Markdown, Javascript, clear_output

# http://pandas-docs.github.io/pandas-docs-travis/user_guide/reshaping.html
import pandas as pd
from pandas import Series, DataFrame
from datetime import datetime, timedelta, timezone
from urllib.parse import urlencode, quote_plus
from requests.utils import requote_uri
import time
import numpy as np
from matplotlib import pyplot as plt
from nimport.utils import tokenize
import json
import os
import calendar as cal

In [ ]:
params = {
    "su": su,
    "start": start,
    "end": end,
    "url": url,
    "baseUrl": baseUrl,
    "service": service,
    "hub": hub
}
root = 'devops-pipelines' if os.path.basename(os.getcwd()) != 'devops-pipelines' else ''
    

In [ ]:
%load_ext Kqlmagic

In [ ]:
# kusto/python utilities
def getTime(timestamp, d):
    return int((cal.timegm(getDateTime(timestamp).timetuple()) + (d * 60)) * 1000);
                
def getDateTime(timestamp):
    s = timestamp[:23] + 'Z' # only allow 5 decimals of precision
    for f in ("%Y-%m-%d %H:%M:%S.%fZ", "%Y-%m-%dT%H:%M:%S.%fZ"):
        try:
            return datetime.strptime(s, f);
        except:
            pass
           
def getKustoQuery(csl_filename, params):

    return tokenize(os.path.join(root, 'queries', csl_filename), params)
                
states = {
    "ja": "ja",
    "healthagent": "healthagent"
};

state = "";

In [ ]:
# Active incidents?
# ActiveIncidents.csl
%kql AzureDataExplorer://tenant="Microsoft.com";code;cluster='Icmcluster';database='IcMDataWarehouse' 
q_activeIncidents = getKustoQuery("ActiveIncidents.csl", params)
q_activeIncidentsResult = %kql -query q_activeIncidents
q_activeIncidentsResultDf = q_activeIncidentsResult.to_dataframe()

In [ ]:
# Note: KQL has a bug where it copies the wrong code (the previous cluster's auth code instead of current) when we try to auth to multiple clusters
# Copy the code manually instead, if you already closed the dialog, copy code and go to https://microsoft.com/devicelogin
%kql AzureDataExplorer://tenant="Microsoft.com";code;cluster='VSO';database='VSO'

In [ ]:
# LocationName.csl
q_loc = getKustoQuery("LocationName.csl", params)
locationNameResult = %kql -query q_loc
locationName = locationNameResult[0]['Tenant']
params["locationName"] = locationName

In [ ]:
# AffectedAccounts.csl
q_affectedAccounts = getKustoQuery("delays/AffectedAccounts.csl", params)
q_affectedAccountsResult = %kql -query q_affectedAccounts
q_affectedAccountsResultDf = q_affectedAccountsResult.to_dataframe()

In [ ]:
# Abusers.csl
q_abusers = getKustoQuery("delays/Abusers.csl", params)
Abusers = %kql -query q_abusers
abusersDf = Abusers.to_dataframe();
finalabusersList = np.intersect1d(q_affectedAccountsResultDf["HostId"].values, abusersDf["HostId"].values);

In [ ]:
# AffectedAccounts.csl
q_affAccounts = getKustoQuery("delays/AffectedAccounts.csl", params)
q_affAccounts_r = %kql -query q_affAccounts
q_affAccounts_df = q_affAccounts_r.to_dataframe();

In [ ]:
# DelayedAccountsAreAbusers.csl
q_delayedAccountsAreAbusers = getKustoQuery("delays/DelayedAccountsAreAbusers.csl", params)
q_delayedAccountsAreAbusers_r = %kql -query q_delayedAccountsAreAbusers
q_delayedAccountsAreAbusers_df = q_delayedAccountsAreAbusers_r.to_dataframe()

In [ ]:
# WhatChanged.csl
q_whatChanged = getKustoQuery("WhatChanged.csl", params)
q_whatChanged_r = %kql -query q_whatChanged
q_whatChanged_df = q_whatChanged_r.to_dataframe();

In [ ]:
# HealthAgentActions.csl
q_haActions = getKustoQuery("HealthAgentActions.csl", params)
q_haActions_r = %kql -query q_haActions
q_haActions_df = q_haActions_r.to_dataframe();

In [ ]:
# VIPSwap.csl
q_vipSwap = getKustoQuery("VIPSwap.csl", params)
vipSwapResult =  %kql -query q_vipSwap
vipSwapResultDf = vipSwapResult.to_dataframe()

In [ ]:
# WhatDelayed.csl
q_whatDelayed = getKustoQuery("delays/WhatDelayed.csl", params)
q_whatDelayedResult = %kql -query q_whatDelayed
q_whatDelayedResultDf = q_whatDelayedResult.to_dataframe()

In [ ]:
# Load.csl
q_load = getKustoQuery("delays/Load.csl", params)
q_loadResult = %kql -query q_load
q_loadResultDf = q_loadResult.to_dataframe()

In [ ]:
# https://plot.ly/python/graphing-multiple-chart-types/

# https://plot.ly/python/line-and-scatter/
import plotly.graph_objs as go
import plotly
plotly.offline.init_notebook_mode(connected=True)

In [ ]:
# visualize delays
delays = go.Scatter(
    x=q_affAccounts_df["PreciseTimeStamp"],
    y=q_affAccounts_df["MessageDelayInSeconds"],
    mode = 'lines',
    name = 'Delays in seconds',
    text= q_affAccounts_df['Name']
)

changed = go.Scatter(
    x=q_whatChanged_df["TIMESTAMP"],
    y=np.repeat(50, len(q_whatChanged_df["TIMESTAMP"].values)),
    mode = 'lines+markers',
    name = 'What Changed',
    text = q_whatChanged_df["Name"],
    marker=dict(
        size=32,
        color = np.random.randn(500),
        colorscale='Viridis'
    )
)

mitigations = go.Scatter(
    x=q_haActions_df["PreciseTimeStamp"],
    y=np.repeat(50, len(q_haActions_df["PreciseTimeStamp"].values)),
    mode = 'markers',
    name = 'Mitigations',
    text = q_haActions_df[["MitigationName", "RoleInstance"]].apply(lambda x: ''.join(x), axis=1),
    marker = dict(
        size = 10,
        color = 'rgba(152, 0, 0, .8)',
        line = dict(
            width = 2,
            color = 'rgb(0, 0, 0)'
        )
    )
)

data = [delays, changed, mitigations]
plotly.offline.iplot(data)

In [ ]:
print('=' * 50)
print('Report!')
print('=' * 50, '\n\n')

jarvisParams = {'su': su, 'start': getTime(start, -10), 'end': getTime(end, 10), 'service': service }

# jarvis
jarvisLink = """https://jarvis-west.dc.ad.msft.net/dashboard/VSO-ServiceInsights/DevOpsReports/TFS DevOpsReports""" \
    """?overrides=[{"query":"//*[id='Service']","key":"value","replacement":"%(service)s"},""" \
    """{"query":"//*[id='RoleInstance']","key":"value","replacement":""},""" \
    """{"query":"//*[id='ScaleUnit']","key":"value","replacement":"%(su)s"}]""" \
    """&globalStartTime=%(start)s&globalEndTime=%(end)s&pinGlobalTimeRange=true""" % jarvisParams;
print('Jarvis dashboard link:\n', requote_uri(jarvisLink), '\n')

# what changed? analysis
print()
print('What changed? =============================')
if(len(q_whatChanged_df.index) == 0):
    print("No relevant changes found...")
else:
    up_prefix = "";
    mit_prefix = "";
    text = "";
    for index, row in q_whatChanged_df.iterrows():
        if(row.title.lower().find('upgrade') != -1):
            if not up_prefix:
                up_prefix += "Looks like, there's upgrade in progress...\n\n";
            text += """%s %s %s \n""" % (row.TIMESTAMP, row.title, row.buildNumber);
        if(row.title.lower().find('mitigation') != -1):
            if not mit_prefix:
                mit_prefix += "Looks like, there are some mitigations by health agent...\n\n";
                state += states["healthagent"];
            text += """%s %s %s""" % (row.TIMESTAMP, row.title, row.buildNumber);
            
    if text:
        print(up_prefix + mit_prefix + text)
    else:
        print(q_whatChanged_df)
        
# active incidents?
print()
print('Active incidents? =============================')
otherIncidentsCount = 0;
for index, row in q_activeIncidentsResultDf.iterrows():
    if(row.Title.find("Kalypso: Build Orchestrator Delays ICM") == -1):
        otherIncidentsCount+=1;
        
if(otherIncidentsCount > 0):
    print("We found some incidents during the time period, check if they are related...")
    # styling
    def make_clickable(url, text):
        return '{0}'.format(url)

    newDf = q_activeIncidentsResultDf.assign(URL=[*map(lambda x: make_clickable("""https://icm.ad.msft.net/imp/v3/incidents/details/%s/home""" % (x), "ICMLink"), q_activeIncidentsResultDf.IncidentId)])
    print("ICM link to copy - " + "https://icm.ad.msft.net/imp/v3/incidents/details/INCIDENTID/home")
    print(newDf[['IncidentId','Severity','Title']])
else:
    print("No active incidents that could be related are found...")

print()
print('Queue Load =============================')
ar = q_loadResultDf[q_loadResultDf["Name"] == "DTPlanQueued"].values[:, 2]
queuedGreatherThan100 = np.where(ar > 100)
if len(queuedGreatherThan100[0]) > 0:
    print("""More than 100 requests are queued in 1 minute (Actual: %s)...could be a load issue""" % (np.amax(ar)))
else: 
    print('...everything looks good?')
# ja load
print()
print('JA Load =============================')
q_whatDelayedResultPendingJobsDf = q_whatDelayedResultDf[q_whatDelayedResultDf.Pivot == "\JobService(_Total)\Total Pending Jobs"]
pendingGreaterThan10Result = np.where(q_whatDelayedResultPendingJobsDf.avg_CounterValue.values > 10)
if len(pendingGreaterThan10Result[0]) > 0:
    print("We are seeing high pending jobs from job agent (highest being %s total pending jobs in 1 min), could be an issue with job agents..." % (np.max(q_whatDelayedResultPendingJobsDf.avg_CounterValue.values)))    
    # update state
    state += states["ja"]
    
    open_nb(os.path.join(root, 'ja.ipynb'), params, redirect=False)
    jaUrl = baseUrl + "/devops-pipelines/ja.ipynb"
    print('Investigate job agent related issues by going here:', requote_uri(jaUrl), '\n')

    jaJarvisLink = """https://jarvis-west.dc.ad.msft.net/dashboard/VSO-ServiceInsights/PlatformViews/Compute-JA""" \
    """?overrides=[{"query":"//*[id='Service']","key":"value","replacement":"%(service)s"},""" \
    """{"query":"//*[id='RoleInstance']","key":"value","replacement":""},""" \
    """{"query":"//*[id='ScaleUnit']","key":"value","replacement":"%(su)s"}]""" \
    """&globalStartTime=%(start)s&globalEndTime=%(end)s&pinGlobalTimeRange=true""" % jarvisParams;
    print('Jarvis dashboard link for job agents:\n', requote_uri(jaJarvisLink), '\n')
else:
    print('...everything looks good?')
    
# abuse detection?
print()
print('What users are impacted? =============================')
if len(finalabusersList) > 0:
    print('Found abusers - !!')
print(q_delayedAccountsAreAbusers_df)
        
#
# vip swap
print()
print('Vip Swap? =============================')
if len(vipSwapResultDf.index) > 0:
    viptime = vipSwapResultDf["TIMESTAMP"][0]
    starttime = getDateTime(start)
    delta = starttime.replace(tzinfo=None) - viptime.replace(tzinfo=None)
    if delta.total_seconds() > 0:
        print("""VIP SWAP happened: %s days %s hours %s minutes ago (%s) (issue start: %s)""" % (delta.days, delta.seconds//3600, (delta.seconds//60) % 60, viptime, start))
    else:
        print('...no swaps recorded in the given time range')
else:
    print('...no swaps recorded in the given time range')

# more analysis?   
print()
print('More analysis =============================')

url = baseUrl + "/devops-pipelines/sla.ipynb"
SLAParams = {
"triggerTime": params["start"],
"scaleUnit": params["su"],
"service": params["service"]
}
open_nb(os.path.join(root, 'sla.ipynb'), SLAParams, redirect=False)
print('Investigate SLA by going here:', requote_uri(url), '\n') 

url = baseUrl + "/devops-pipelines/impact.ipynb"
open_nb(os.path.join(root, 'impact.ipynb'), params, redirect=False)
print('Investigate any existing customer impact by going here:', requote_uri(url), '\n') 